# Answer 5

In the script below constraints are defined and placed on the columns in the features data. Any violation of these constraints are added to an error log and printed at the end. In this way every data value can be checked against what is expected.

In [1]:
import numpy as np
import pandas as pd
import re

**Read in data**

In [2]:
features = pd.read_excel("../../data/question_005/SGD-Features.xlsx")

**Define function which takes a column and its constraints and returns an error log**

In [3]:
def enforce_constraints(rule):
    """
    Create a list of entries that violate
    the specified constraints.
    """
    
    column = rule['column']
    rows = features['Row'].values
    rules = rule['constraints']
    
    if column != 'All':
        data = features[rule['column']]
    
    error_log = []
    
    if 'not_null' in rules:
        for i, y in enumerate(data.isnull()):
            if y:
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'null value'
                    })
    
    if 'no_whitespace' in rules:
        pattern = re.compile('^\s|.+\s$')
        for i, y in enumerate(data):
            if pattern.match(str(y)):
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'leading/lagging whitespace'
                    })
    
    if 'is_unique' in rules:
        for i, y in enumerate(data.duplicated()):
            if y:
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'duplicate value'
                    })
    
    if 'is_digit' in rules:
        for i, y in enumerate(data):
            try:
                float(y)
            except ValueError:
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': "nonnumeric value"
                    })
    
    if 'is_consecutive' in rules:
        for i, y in enumerate(data):
            if i == 0:
                continue
            last_value = data.values[i - 1]
            current_value = data.values[i]
            if current_value - last_value != 1:
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'nonconsecutive value'
                    })
    
    if 'is_orf' in rules:
        for i, y in enumerate(data):
            if y != "ORF":
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'non-ORF value'
                    })
            
    
    if 'is_gene_name' in rules:
        # Looks for three letters plus a number
        pattern = re.compile('[a-zA-Z]{3}\d+')
        for i, y in enumerate(data):
            if pd.isnull(y):
                continue
            if not pattern.match(str(y)):
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'non-gene value'
                    })
    
    if 'is_higher_than_stop' in rules:
        stops = features['Stop coordinate'].values
        for i, y in enumerate(data):
            if stops[i] <= y:
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'higher value than the stop coordinate'
                    })
    
    if 'is_w_or_c' in rules:
        for i, y in enumerate(data):
            if y != 'W' and y != 'C':
                 error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'non-W or C value'
                    })
    
    if 'is_yyyymmdd_format' in rules:
        pattern = re.compile('\d{4}-\d{2}-\d{2}')
        for i, y in enumerate(data):
            if not pattern.match(str(y)):
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'non-date format'
                    })
    
    if 'is_sentence' in rules:
        for i, y in enumerate(data):
            if pd.isnull(y):
                continue
            if len(str(y).split(" ")) < 3:
                error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'non-sentence description'
                    })
                
    if 'right_shift' in rules:
        # Check if any rows are right shifted
        if features.shape[1] > 12:
            data = features.iloc[:,12:13].values
            for i, y in enumerate(data):
                if not pd.isnull(y):
                    error_log.append({
                        'column': column,
                        'row': rows[i],
                        'problem': 'right-shifted row'
                    })
            
        
    return error_log

**Define constraints according to expected values (and also check for common errors)**

In [4]:
# Bind constraints to columns
rules = [
    {
        'column': 'Row',
        'constraints': ['not_null',
                        'no_whitespace',
                        'is_consecutive']
    },
    {
        'column': 'Primary SGDID',
        'constraints': ['not_null',
                        'no_whitespace',
                        'is_unique']
    },
    {
        'column': 'Feature type',
        'constraints': ['not_null',
                        'no_whitespace',
                        'is_orf']
    },
    {
        'column': 'Feature name',
        'constraints': ['no_whitespace']
    },
    {
        'column': 'Gene name',
        'constraints': ['no_whitespace',
                        'is_gene_name']
    },
    {
        'column': 'Secondary SGDID',
        'constraints': ['no_whitespace']
    },
    {
        'column': 'Chromosome',
        'constraints': ['no_whitespace',
                        'is_digit']
    },
    {
        'column': 'Start coordinate',
        'constraints': ['not_null',
                        'no_whitespace',
                        'is_digit',
                        'is_higher_than_stop']
    },
    {
        'column': 'Stop coordinate',
        'constraints': ['not_null',
                        'no_whitespace',
                        'is_digit']
    },
    {
        'column': 'Strand',
        'constraints': ['not_null',
                        'no_whitespace',
                        'is_w_or_c']},
    {
        'column': 'Sequence version date',
        'constraints': ['no_whitespace',
                        'is_yyyymmdd_format']},
    {
        'column': 'Description',
        'constraints': ['no_whitespace',
                        'is_sentence']
    },
    {
        'column': 'All',
        'constraints': ['right_shift']
    }
]


**Print error log**

In [5]:
for column in rules:
    errors = enforce_constraints(column)
    if errors:
        print("In the {} column:".format(column['column']))
        for error in errors:
            print("Row {} has a {}.".format(error['row'], error['problem']))
        print("")

In the Row column:
Row 7 has a nonconsecutive value.
Row 15 has a nonconsecutive value.
Row 28 has a nonconsecutive value.
Row 67 has a nonconsecutive value.
Row 75 has a nonconsecutive value.
Row 111 has a nonconsecutive value.
Row 135 has a nonconsecutive value.
Row 147 has a nonconsecutive value.

In the Primary SGDID column:
Row 114 has a null value.
Row 97 has a duplicate value.

In the Feature type column:
Row 35 has a non-ORF value.
Row 57 has a non-ORF value.

In the Feature name column:
Row 42 has a leading/lagging whitespace.
Row 81 has a leading/lagging whitespace.

In the Gene name column:
Row 55 has a non-gene value.
Row 59 has a non-gene value.
Row 95 has a non-gene value.

In the Start coordinate column:
Row 7 has a higher value than the stop coordinate.
Row 8 has a higher value than the stop coordinate.
Row 15 has a higher value than the stop coordinate.
Row 18 has a higher value than the stop coordinate.
Row 30 has a higher value than the stop coordinate.
Row 31 has a 

**Display results as table**

In [6]:
errors = []
for rule in rules:
    errors.append(enforce_constraints(rule))
results = pd.DataFrame([item for sublist in errors for item in sublist])
results

,column,problem,row
0,Row,nonconsecutive value,7
1,Row,nonconsecutive value,15
2,Row,nonconsecutive value,28
3,Row,nonconsecutive value,67
4,Row,nonconsecutive value,75
5,Row,nonconsecutive value,111
6,Row,nonconsecutive value,135
7,Row,nonconsecutive value,147
8,Primary SGDID,null value,114
9,Primary SGDID,duplicate value,97


**Export to TSV**

In [7]:
results.to_csv("error_results.tsv", sep='\t')